In [1]:
import subprocess as s
import os
import string
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
import warnings
from jiwer import wer
import numpy as np
import time
import pickle
warnings.filterwarnings('ignore')

In [2]:
tag = "Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best"
d = ModelDownloader('Model')
d.download(tag)

'/home/iemmanuelvm/Documents/ASR_ESPNET_LIBRESPEECH/Model/653d10049fdc264f694f57b49849343e/asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip'

In [3]:
speech2text = Speech2Text(
    **d.unpack_local_file('Model/653d10049fdc264f694f57b49849343e/asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best.zip'),
    device="cuda:0", #cuda if gpu
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

In [4]:
# Strips text of punctuation and makes it uppercase
def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [5]:
# Generates and returns transcript given audio file path
def get_transcript(path):
    speech, rate = soundfile.read(path)
    nbests = speech2text(speech)
    text, *_ = nbests[0]
    return text, rate

In [6]:
def get_metrics_wer(PATH):
    WER = []
    # Set necessary paths
    path = os.path.join(os.getcwd(), PATH)
    files = os.listdir(path+'/Audios')
    # For every file in audio directory
    for file in files:
        
        
        
        max_attempts = 10
        attempts = 0
        success = False

        while attempts < max_attempts and not success:
            try:
                # Código que puede generar una excepción
                # ...
                
                # Get transcript, converting to .wav if not a wav
                if not file.endswith('.flac'):
                    os.chdir(path+'/Audios')
                    s.run(f"ffmpeg -i {file} {file.split('.')[0]}.flac", shell=True, check=True, universal_newlines=False)
                    os.chdir('../..')
                    file = file.split('.')[0]+'.flac'
                    text, est_rate = get_transcript(f'{path}/Audios/{file}')
                    os.remove(f'{path}/Audios/{file}')
                else:
                    text, est_rate = get_transcript(f'{path}/Audios/{file}')

                # Fetch true transcript
                label_file = file.split('.')[0]+'.txt'
                with open(f'{path}/Textos/{label_file}', 'r') as f:
                    true_text = f.readline()
                # Print true transcript and hypothesis
                error = wer(true_text, text_normalizer(text))
                WER.append(error)
                print(error)
                print(file)
                #print(f"Reference text: {true_text}")
                #print(f"ASR hypothesis: {text_normalizer(text)}\n\n")

                # Si el código se ejecuta correctamente, establece success en True
                success = True

            except Exception as e:
                # Captura la excepción y muestra un mensaje de error
                print(f"Se produjo un error: {e}")

                # Incrementa el contador de intentos
                attempts += 1

        if success:
            print("El código se ejecutó correctamente.")
        else:
            print("No se pudo ejecutar el código después de varios intentos.")
        
        
        
        
        
        
        
    return WER

In [7]:
#BASE-LINE
#start_time = time.time()
#WER_CLEAN = get_metrics_wer('egs/folders_aux/audio_clean/')
#end_time = time.time()
#total_time = end_time - start_time
#print("El código tardó {} segundos en ejecutarse".format(total_time))
#print(np.sum(np.array(WER_CLEAN))/len(np.array(WER_CLEAN)))

In [8]:
#WERS=[]
path_denoised_main = 'egs/folders_aux/audio_with_noise/'
for folder_db in sorted(os.listdir(path_denoised_main)):
    print(folder_db)
    for folder_noise in sorted(os.listdir(os.path.join(path_denoised_main, folder_db))):
        print(os.path.join(path_denoised_main, folder_db, folder_noise))
        name = os.path.join(path_denoised_main, folder_db, folder_noise).split('/')
        print(str(name[4])+'_'+str(name[3])+'db.pkl')
        start_time = time.time()
        print(os.path.join(path_denoised_main, folder_noise))
        WER = get_metrics_wer(os.path.join(path_denoised_main, folder_db, folder_noise))
        end_time = time.time()
        total_time = end_time - start_time
        print("El código tardó {} segundos en ejecutarse".format(total_time))
        AVG_WER = np.sum(np.array(WER))/len(np.array(WER))
        with open('shared/'+str(name[4])+'_'+str(name[3])+'db_noise.pkl', 'wb') as archivo:
            pickle.dump(AVG_WER, archivo)
            archivo.close()
        print('AVG WER: '+str(AVG_WER))
        #break
    #break
        #WERS.append(AVG_WER)

-3
egs/folders_aux/audio_with_noise/-3/nonstationary_babycry
nonstationary_babycry_-3db.pkl
egs/folders_aux/audio_with_noise/nonstationary_babycry
0.375
1867-148436-0039.flac
El código se ejecutó correctamente.
0.7575757575757576
1898-145720-0003.flac
El código se ejecutó correctamente.
0.5
1963-142393-0001.flac
El código se ejecutó correctamente.
0.45652173913043476
1867-148436-0017.flac
El código se ejecutó correctamente.
0.71875
19-227-0023.flac
El código se ejecutó correctamente.
0.5263157894736842
1737-142397-0032.flac
El código se ejecutó correctamente.
0.358974358974359
196-122152-0006.flac
El código se ejecutó correctamente.
0.4523809523809524
1926-147979-0041.flac
El código se ejecutó correctamente.
0.5405405405405406
1898-145715-0003.flac
El código se ejecutó correctamente.
0.5454545454545454
1926-143879-0034.flac
El código se ejecutó correctamente.
0.8478260869565217
1898-145720-0006.flac
El código se ejecutó correctamente.
0.3469387755102041
1723-141149-0016.flac
El código 

In [10]:
#WERS=[]
path_denoised_main = 'egs/folders_aux/audios_denoised_dnn/'
for folder_db in sorted(os.listdir(path_denoised_main)):
    print(folder_db)
    for folder_noise in sorted(os.listdir(os.path.join(path_denoised_main, folder_db))):
        print(os.path.join(path_denoised_main, folder_db, folder_noise))
        name = os.path.join(path_denoised_main, folder_db, folder_noise).split('/')
        print(str(name[4])+'_'+str(name[3])+'db.pkl')
        start_time = time.time()
        print(os.path.join(path_denoised_main, folder_noise))
        WER = get_metrics_wer(os.path.join(path_denoised_main, folder_db, folder_noise))
        end_time = time.time()
        total_time = end_time - start_time
        print("El código tardó {} segundos en ejecutarse".format(total_time))
        AVG_WER = np.sum(np.array(WER))/len(np.array(WER))
        with open('shared2/'+str(name[4])+'_'+str(name[3])+'db_dnn.pkl', 'wb') as archivo:
            pickle.dump(AVG_WER, archivo)
            archivo.close()
            #WERS.append(AVG_WER)
        print('AVG WER: '+str(AVG_WER))
        #break
    #break

-3
egs/folders_aux/audios_denoised_dnn/-3/nonstationary_babycry
nonstationary_babycry_-3db.pkl
egs/folders_aux/audios_denoised_dnn/nonstationary_babycry
0.20833333333333334
1867-148436-0039.flac
El código se ejecutó correctamente.
0.3333333333333333
1898-145720-0003.flac
El código se ejecutó correctamente.
0.07142857142857142
1963-142393-0001.flac
El código se ejecutó correctamente.
0.21739130434782608
1867-148436-0017.flac
El código se ejecutó correctamente.
0.4375
19-227-0023.flac
El código se ejecutó correctamente.
0.15789473684210525
1737-142397-0032.flac
El código se ejecutó correctamente.
0.1282051282051282
196-122152-0006.flac
El código se ejecutó correctamente.
0.23809523809523808
1926-147979-0041.flac
El código se ejecutó correctamente.
0.05405405405405406
1898-145715-0003.flac
El código se ejecutó correctamente.
0.36363636363636365
1926-143879-0034.flac
El código se ejecutó correctamente.
0.3695652173913043
1898-145720-0006.flac
El código se ejecutó correctamente.
0.040816326

In [11]:
#WERS=[]
path_denoised_main = 'egs/folders_aux/audios_denoised_wt/'
for folder_db in sorted(os.listdir(path_denoised_main)):
    print(folder_db)
    for folder_noise in sorted(os.listdir(os.path.join(path_denoised_main, folder_db))):
        print(os.path.join(path_denoised_main, folder_db, folder_noise))
        name = os.path.join(path_denoised_main, folder_db, folder_noise).split('/')
        print(str(name[4])+'_'+str(name[3])+'db.pkl')
        start_time = time.time()
        print(os.path.join(path_denoised_main, folder_noise))
        WER = get_metrics_wer(os.path.join(path_denoised_main, folder_db, folder_noise))
        end_time = time.time()
        total_time = end_time - start_time
        print("El código tardó {} segundos en ejecutarse".format(total_time))
        AVG_WER = np.sum(np.array(WER))/len(np.array(WER))
        with open('shared3/'+str(name[4])+'_'+str(name[3])+'db_dnn_wt.pkl', 'wb') as archivo:
            pickle.dump(AVG_WER, archivo)
            archivo.close()
            #WERS.append(AVG_WER)
        print('AVG WER: '+str(AVG_WER))
        #break
    #break

-3
egs/folders_aux/audios_denoised_wt/-3/nonstationary_babycry
nonstationary_babycry_-3db.pkl
egs/folders_aux/audios_denoised_wt/nonstationary_babycry
0.20833333333333334
1867-148436-0039.flac
El código se ejecutó correctamente.
0.36363636363636365
1898-145720-0003.flac
El código se ejecutó correctamente.
0.14285714285714285
1963-142393-0001.flac
El código se ejecutó correctamente.
0.1956521739130435
1867-148436-0017.flac
El código se ejecutó correctamente.
0.28125
19-227-0023.flac
El código se ejecutó correctamente.
0.10526315789473684
1737-142397-0032.flac
El código se ejecutó correctamente.
0.05128205128205128
196-122152-0006.flac
El código se ejecutó correctamente.
0.14285714285714285
1926-147979-0041.flac
El código se ejecutó correctamente.
0.02702702702702703
1898-145715-0003.flac
El código se ejecutó correctamente.
0.0
1926-143879-0034.flac
El código se ejecutó correctamente.
0.45652173913043476
1898-145720-0006.flac
El código se ejecutó correctamente.
0.061224489795918366
1723-